In [1]:
import napari
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
# Part from this notebook has been adapted from Robert Haase's material (which is another great source 
# and good to have for future references):
# https://haesleinhuepf.github.io/BioImageAnalysisNotebooks/intro.html

In [2]:
v = napari.Viewer()

### Open the built-in 'human_mitosis' image and create some layers

In [3]:
# Save the numpy array 
mitosis_image = v.layers['human_mitosis'].data

In [4]:
points_layer = v.layers['Points'].data

In [5]:
# Create a points layer and quantify distances:
def distance(points):
    dist = np.sqrt(np.sum((points[0] - points[1])**2, axis=0))
    return dist

In [6]:
print('Long-axis mitotic cell distance:')
print(distance(points_layer))

Long-axis mitotic cell distance:
9.738928950453042


In [7]:
# Exercise: save the points layer in disk as a numpy array and
# add the points as a points layer within this notebook from the saved file.
# Hint: search for np.save() and v.add_points()

Do some annotations using a 'labels' layer

In [8]:
# Explore intensity values for different states:
label_dict = {1: 'mitotic', 2: 'non-mitotic', 3: 'background'}

for label_idx, label_name in label_dict.items():
    pixel_values = v.layers['human_mitosis'].data[v.layers['Labels'].data == label_idx]
    print(f'{label_name} values')
    print(f'Mean intensity: {np.mean(pixel_values)}')
    print(f'Standard deviation: {np.std(pixel_values)}')
    print('-'*20)

mitotic values
Mean intensity: 237.33333333333334
Standard deviation: 1.649915822768611
--------------------
non-mitotic values
Mean intensity: 83.07692307692308
Standard deviation: 14.376591756481936
--------------------
background values
Mean intensity: 12.481012658227849
Standard deviation: 0.8246327835393585
--------------------


### Using the labels layer to train a pixel classifier
We will use the [Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to see an easy usage for layers manipulation in napari. 

In [10]:
# Instantiate np.array with manual annotations:
manual_labels = v.layers['Labels'].data

In [11]:
# For a random forest pixel classifier we can use the manual anotations previously created and 
# increase the feature space using a set of filters.
# Import our custom functions:
from filter_functions import augment_feature_stack, format_data

In [12]:
feature_stack = augment_feature_stack(mitosis_image)

In [13]:
# Exercise: visualize in the napari viewer the filtered images.
# Hint: check how to access and reshape a numpy array

In [14]:
X, y = format_data(feature_stack, manual_labels)

print("input shape", X.shape)
print("annotation shape", y.shape)

input shape (117, 3)
annotation shape (117,)


In [15]:
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X, y)

RandomForestClassifier(max_depth=2, random_state=0)

In [16]:
res = classifier.predict(feature_stack.T) - 1 # we subtract 1 to make background = 0

In [20]:
# Exercise: visualize results in napari
# Hint: Use the v.add_labels() method for the napari viewer

# Additional exercise:
# Improve the algorithm predictions